# Lab CMBLenses

## Model

In [1]:
] activate "/home/jaimerz/julia_repos/CMBLensingInferenceTestProblem"

  Activating project at `~/julia_repos/CMBLensingInferenceTestProblem`


In [2]:
using CMBLensing, CMBLensingInferenceTestProblem, CUDA, LaTeXStrings, MCMCDiagnosticTools, Plots, ProgressMeter, Random, Zygote
ENV["LINES"] = 10;

In [3]:
using Revise
using MCHMC

[ Info: Precompiling MCHMC [92da4d09-9da0-4f8a-8b31-da41ef0ab6ee]


In [4]:
prob = load_cmb_lensing_problem(storage = Array,
                                T = Float32,
                                Nside = 16);

MAP_joint: 100%|████████████████████████████████████████| Time: 0:00:55
  step:        20
  logpdf:      6120.72
  α:           0.53812873
  CG:          2 iterations
  Linesearch:  12 bisections


In [5]:
target = CMBLensingTarget(prob)

CMBLensingTarget(770, ComplexF64[0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 4.435770034790039 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 1.0 + 0.0im], MCHMC.var"#nlogp#25"{CMBLensingInferenceTestProblem.CMBLensingLogDensityProblem}(CMBLensingInferenceTestProblem.CMBLensingLogDensityProblem(BaseDataSet(d, Cf, Cn, Cn̂, M, M̂, B, B̂, logprior, Cϕ, Cf̃, D, G, L, Nϕ), [-9.376073837280273, -6.5940842628479, -5.405848979949951, -7.392019748687744, -8.028088569641113, -5.530062198638916, -1.2073196172714233, 1.3572702407836914, 3.625626564025879, 5.442736625671387  …  3.239753709749493e-8, -1.2966495432920055e-8, -8.702781428837625e-8, -1.6196999297335424e-7, -2.175492284095526e-7, -2.6352793724981893e-7, -3.134788357783691e-7, -3.52711253981397e-7, -1.6094379124341003, 0.0], [-9.446122169494629, -8.039124488830566, -5.012213706970215, -7.389093399047852, -7.862917423248291, -5.085802555084

## Sample

In [6]:
eps=0.005
L=sqrt(2)
spl=Sampler(;eps=eps, L=L)

Sampler(Settings(MersenneTwister(0), 0.005, 1.4142135623730951, 0.0, 0.1931833275037836, 0.0005, 2000, 1000, 10, "LF"), MCHMC.Leapfrog)

In [9]:
samples = Sample(spl, target; num_steps=10000, initial_x=prob.Ωstart);

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 16 and 770

## Plotting

In [8]:
Plots.default(fmt=:png, dpi=120, size=(500,300), legendfontsize=10)

In [16]:
plot([exp(Ω.θ.r) for (Ω,) in chain], label=L"r", xlabel="step")
plot!([exp(Ω.θ.Aϕ) for (Ω,) in chain], label=L"A_\phi")

LoadError: UndefVarError: `chain` not defined

In [17]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [18]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [19]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [20]:

ps = map([(:ϕ°,:I,L"L",L"\phi^\circ"), (:f°,:E,"L\ell",L"E^\circ"), (:f°,:B,L"\ell",L"B^\circ")]) do (k1, k2, xlabel, title)
    plot(get_Cℓ(prob.Ωtrue[k1][k2]); label="true", xlabel, title)
    plot!(get_Cℓ(prob.Ωstart[k1][k2]); label="start", xlabel, title)
    plot!(get_Cℓ(chain[end][1][k1][k2]); label="last sample", xlabel, title)
end
plot(ps..., layout=(1,3), xscale=:log10, yscale=:log10, size=(1000,300), legend=:bottomleft)

LoadError: UndefVarError: `prob` not defined